# new 方法: stage1 & stage2

In [ ]:
import sys
sys.path.append("..")

import pandas as pd
from models import Yolov4
from tensorflow import keras
from config import yolo_config
from iou import get_iou, calculate_ciou

import numpy as np
import seaborn as sns
import matplotlib.pylab as pl
import matplotlib.pyplot as plt

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
# from sklearn.metrics import PrecisionRecallDisplay
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import ConfusionMatrixDisplay

import math
import cv2 as cv

# import pickle

import os
# 使用第二張 GPU 卡
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# 載入模型

In [ ]:
# yolo_config['score_threshold']=0.7
# print(yolo_config)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
class_name_path = '../class_names/classes.txt'
model = Yolov4(
               weight_path    =None, 
               config         =yolo_config,
               class_name_path=class_name_path, 
               )
model.load_weights(path="../model/202303/20230403_sopbox_0.weights")            # 換stage2模型!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

#22 1102是沒有10%背
#22 1104是有10%背景
#22 0104目前表現最佳

#186 0330是沒有10%背
#186 0331是有10%背景

# 每一個test image "True" bounding box

In [ ]:
true_box_PATH = '../test_dataset/txt/20230330_anno_test_lines_0.txt'
# true_box_PATH = '../test_dataset/txt/20221102_anno_test_lines_0.txt'       # 這邊要修改train/test 的路徑!!!!!!!!
# true_box_PATH = '../dataset/txt/20221102_anno_train_lines_0.txt'
# true_box_PATH = '../dataset/txt/20221102_anno_val_lines_0.txt'


true_bboxes_names = []
true_bboxes_list  = []
with open(true_box_PATH) as f:
    for line in f.readlines():
        true_bboxes_list.append(tuple([int(i) for i in line.split(' ')[1].split(',')[:4]]))
        true_bboxes_names.append(line.split(' ')[0])

## 整理成Dataframe
true_bboxes = pd.DataFrame({'test_name':true_bboxes_names, 
                            'true_bbox':true_bboxes_list
                            })
true_bboxes   

# 每一個test image "Pred" bounding box

In [ ]:
# train = ['a', 'b', 'c', 'e', 'f', 'g', 'j', 'm', 'n', 'p', 'r', 's', 't', 'u']
# val   = ['d', 'i', 'k', 'o']
# test  = ['h', 'l', 'q', 'z']


PATH = '../../classification/dataset_smooth_186video_20230323/test/1/'
# PATH = '../../classification/dataset_186video_20230323/test/1/'

# PATH = '../../classification/dataset_smooth_22video_20221031/test/1/'                #!!!!!!!!!!!!!!!!!!!!!!
# PATH = '../../classification/dataset_22video_20221031/test/1/'
# PATH = '../../classification/dataset_smooth_22video_20221031/train/1/'            # 這邊要修改train/test 的路徑!!!!!!!!
# PATH = '../../classification/dataset_smooth_22video_20221031/validation/1/'       # 這邊要修改train/test 的路徑!!!!!!!!

pred_bboxes_names  = []  # all image names
pred_bboxes_scores = []  # all bboxes scores
pred_bboxes_list   = []  # all bboxes
for test_img_name in true_bboxes['test_name']:  # glob(os.path.join(PATH, '*.png')):
    test_img_path = PATH + test_img_name
    output_img, detections = model.predict(img_path=test_img_path, 
                                        random_color=False, # 框顏色
                                        plot_img=False, # 顯示出框選圖片
                                        show_text=True, # 框上面加上類別文字
                                        box_num=1,      # 顯示框的數量
                                        return_output=True)
    
    pred_box_num = len(detections['score'])                
    if pred_box_num > 0:       # 有預測出box
        
        score = detections[0:1]['score'].to_numpy()[0]
        pred_bboxes_names.append(test_img_name)
        pred_bboxes_scores.append(score)
        pred_bboxes_list.append(tuple(detections[0:1][['x1','y1','x2','y2']].to_numpy()[0]))
    else:
        pred_bboxes_names.append(test_img_name)
        pred_bboxes_scores.append(0.0)
        pred_bboxes_list.append('NO')
        print('沒有預測出來有box(label)')

In [ ]:
pred_bboxes = pd.DataFrame({'test_name':pred_bboxes_names, 
                            'pred_bbox':pred_bboxes_list,
                            'score':pred_bboxes_scores
               })
pred_bboxes

# perfect stage2 prediction result

In [ ]:
stage2_pred_result = true_bboxes.merge(pred_bboxes, on=['test_name'], how='left')

In [ ]:
# stage2_pred_result.to_csv("../result_df/test_perfectstage1_stage2_result_20230331.csv", index=False) #!!!!!!!!!!!!!!!!!! stage 3 會用到
stage2_pred_result

# 計算iou 繪圖
參數:
iou_threshold = 0.7    "iou大於0.7才是有框選到"

In [ ]:
def valid(TP, FN, FP, TN):
    
    precision = 0 if (TP+FP)==0 else TP/(TP+FP)
    recall    = 0 if (TP+FN)==0 else TP/(TP+FN)
    f1        = 0 if (precision+recall)==0 else 2*precision*recall/(precision+recall)

    print('precision={:.2f}, recall={:.2f}, f1={:.2f}'.format(precision, recall, f1))
    print(TP,"|",FN,"\n",
          FP,"|",TN)

    return precision, recall, f1

In [ ]:
def plt_precision_recall_curve_1(y_true, y_scores):
    precision, recall, thresholds = precision_recall_curve(y_true, y_scores)
    average_precision             = average_precision_score(y_true, y_scores)
    # 用套件畫
    # disp = PrecisionRecallDisplay(precision=precision, recall=recall, average_precision=average_precision)
    # disp.plot()
    # plt.ylim([0.0, 1.05])
    # plt.xlim([0.0, 1.0])
    # plt.show()

    # 自己畫
    plt.plot(recall, precision, 'b-')   
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])  
    plt.xlabel('recall')
    plt.ylabel('precision')
    plt.title('precision_recall_curve')
    plt.legend(['AP={}'.format(round(average_precision,2))])
    plt.show()

    # print('precision = ', precision)
    # print('recall = ', recall)
    # print('thresholds = ',thresholds)
    print('AP={:.2f}'.format(average_precision))

In [ ]:
def plt_heatmap(con_ma):
    '''
    :input: con_ma:[[TP,FN],[FP,TN]]
    '''
    sns.set(font_scale=1.5)
    f,ax=plt.subplots()
    sns.heatmap(con_ma, annot=True, ax=ax, fmt='d', cmap='YlGnBu', vmax=200, vmin=0) # 畫熱力圖
    ax.set_title('confusion matrix') # 標題
    ax.set_xlabel('predict')         # x軸
    ax.set_ylabel('true')            # y軸
    ax.xaxis.set_ticklabels(['Papilla', 'No Papilla'])
    ax.yaxis.set_ticklabels(['Papilla', 'No Papilla'])
    plt.show()

perfect stage2 測試集結果 固定iou_threshold=0.5
這沒有參考價值，因為要限制score

In [ ]:
# row, col = stage2_pred_result.shape
# iou_thresholds = np.arange(0.5, 0.6, 0.1)      # 固定ciou_threshold=0.5
# # test_img_names = true_bbox.keys()
# precisions, recalls, f1s = [], [], []
# for iou_threshold in iou_thresholds:
#     TP,FP,FN,TN = 0,0,0,0
#     y_true  = []                 # 0:無壺腹乳頭 1:有壺腹乳頭
#     y_score = []                 # confidence分數
#     for r in range(row):                # row: 列數
#         test_img_name = stage2_pred_result.iloc[r:r+1]['test_name'][r]
#         true_bbox     = stage2_pred_result.iloc[r:r+1]['true_bbox'][r]
#         pred_bbox     = stage2_pred_result.iloc[r:r+1]['pred_bbox'][r]
#         score         = stage2_pred_result.iloc[r:r+1]['score'][r]

#         # 限制score<0.5沒有bbox #
#         # if score<0.5:
#         #     pred_bbox = 'NO'
#         ########################


        
#         if true_bbox != 'nan' and pred_bbox != 'NO':
#             ciou = get_iou(true_bbox, pred_bbox)           # 找出真實box和預測box的iou
#             # ciou = calculate_ciou(true_bbox, pred_bbox)                                    # 找出真實box和預測box的ciou
            
#             if ciou >= iou_threshold:   # 如果>=iou，才是有預測到，TP會增加
#                 y_true.append(1)
#                 y_score.append(score)
#                 TP += 1
#             else:
#                 y_true.append(0)
#                 y_score.append(score)
#                 FP += 1                 # 如果<iou，FP也會增加

#         elif true_bbox != 'NO' and pred_bbox == 'NO':
#             y_true.append(1)
#             y_score.append(0)
#             FN += 1

#         elif true_bbox == 'NO' and pred_bbox != 'NO':
#             y_true.append(0)
#             y_score.append(score)
#             FP += 1

#         else:
#             # y_true.append(0)
#             # y_scores.append(0)
#             TN += 1
    
    
#     con_ma = [[TP, FN],[FP, TN]]
#     plt_heatmap(con_ma)                               # 畫熱力圖
#     plt_precision_recall_curve_1(y_true, y_score)     # 畫PR curve
#     precision, recall, f1 = valid(TP, FN, FP, TN)      # P R f1 資訊
#     print('iou_threshold={}'.format(iou_threshold))
#     print("------------------------------------------------------------------------------------")
#     precisions.append(precision)
#     recalls.append(recall)
#     f1s.append(f1)

In [ ]:
def plt_precision_recall_curve_2(iou_thresholds, y_trues, y_scores):
    
    n      = len(y_trues)
    colors = pl.cm.jet(np.linspace(0,1,n))
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])  
    plt.xlabel('recall')
    plt.ylabel('precision')
    plt.title('precision_recall_curve')

    total_average_precision = 0
    for i in range(n):
        iou_threshold = round(iou_thresholds[i],4)
        y_true        = y_trues[i]
        y_score       = y_scores[i]
        average_precision = round(average_precision_score(y_true, y_score), 4)
        print("AP====",average_precision_score(y_true, y_score))
        if average_precision<=0 or math.isnan(average_precision):
            total_average_precision += 0
            plt.plot([0], [0], color=colors[i], label = "IoU={} => AP={}".format(iou_threshold, 0))
        else:
            total_average_precision += average_precision
            precision, recall, thresholds = precision_recall_curve(y_true, y_score)
            # 自己畫
            plt.plot(recall, precision, color=colors[i], label = "IoU={} => AP={}".format(iou_threshold, average_precision))   
        
        # print('precision = ', precision)
        # print('recall = ', recall)
        # print('thresholds = ',thresholds)
        # print('AP={:.2f}'.format(average_precision))

    averaged_AP = round(total_average_precision/n, 4)
    print("Averaged_AP= ", averaged_AP)
    plt.legend(loc='upper left',bbox_to_anchor=(0.0, -0.3))
    plt.show()

# [perfect stage 1] & stage2 
測試集結果 iou_threshold=[0.5:0.05:0.95]

In [ ]:
row, col = stage2_pred_result.shape
iou_thresholds = np.arange(0.5, 1.0, 0.05)      

precisions, recalls, f1s = [], [], []
y_trues  = []
y_scores = []
for iou_threshold in iou_thresholds:
    y_true   = []                # 0:無壺腹乳頭 1:有壺腹乳頭
    y_score  = []                # confidence分數
    for r in range(row):         # row: 列數
        test_img_name = stage2_pred_result.iloc[r:r+1]['test_name'][r]
        true_bbox     = stage2_pred_result.iloc[r:r+1]['true_bbox'][r]
        pred_bbox     = stage2_pred_result.iloc[r:r+1]['pred_bbox'][r]
        score         = stage2_pred_result.iloc[r:r+1]['score'][r]

        if true_bbox != 'NO' and pred_bbox != 'NO':
            ciou = get_iou(true_bbox, pred_bbox)           # 找出真實box和預測box的iou
            # ciou = calculate_ciou(true_bbox, pred_bbox)                     # 找出真實box和預測box的ciou
            
            if ciou >= iou_threshold:   # 如果>=iou，才是有預測到，TP會增加
                y_true.append(1)
                y_score.append(score)   # TP
            else:
                y_true.append(0)
                y_score.append(score)   # FP
        elif true_bbox != 'NO' and pred_bbox == 'NO':   # FN
            y_true.append(1)
            y_score.append(score)
        elif true_bbox == 'NO' and pred_bbox != 'NO':  # FP   因為stage 1 完美，所以不會有這裡的FP
            y_true.append(0)
            y_score.append(score)

    y_trues.append(y_true)
    y_scores.append(y_score)

plt_precision_recall_curve_2(iou_thresholds, y_trues, y_scores)     # 畫PR curve

In [ ]:
stage2_pred_result.head()

# stage1 & stage2 測試結果

### stage1 分類預測結果

In [ ]:
# stage1_pred_result = pd.read_pickle("../../classification/result/stage1_pred_result_train.pickle")   # 這邊換stage1結果!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# stage1_pred_result = pd.read_pickle("../../classification/result/stage1_pred_result_validation.pickle")   # 這邊換stage1結果!!!!!!!!!!!!!!!!!!!!!!!!!!!!


stage1_pred_result = pd.read_pickle("../../classification/result/stage1_pred_result_test_20230323.pickle")   # 這邊換stage1結果!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# stage1_pred_result = pd.read_pickle("../../classification/result/stage1_pred_result_test_nosmooth_20230323.pickle")   # 這邊換stage1結果!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# stage1_pred_result = pd.read_pickle("../../classification/result/stage1_pred_result_test.pickle")   # 這邊換stage1結果!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# stage1_pred_result = pd.read_pickle("../../classification/result/stage1_pred_result_test_nosmooth.pickle")   # 這邊換stage1結果!!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [ ]:
filter1 = stage1_pred_result['test_true']!=0 
filter2 = stage1_pred_result['stage1_test_pred']!=0
stage1_pred_result_all = stage1_pred_result[(filter1 | filter2)]
print(stage1_pred_result_all.shape)
stage1_pred_result_all.head(2)

In [ ]:
'''這6個(FP)score，不能是0，因為FP要用到score，所以要預測bbox & score'''

filter3 = stage1_pred_result['test_true']==0 
filter4 = stage1_pred_result['stage1_test_pred']==1
fp_temp_df = stage1_pred_result[(filter3 & filter4)]

row_, col_ = fp_temp_df.shape
print("這{}個(FP)score，不能是0，因為FP要用到score，所以要預測bbox & score".format(row_))

pred_bboxes_names_  = []
pred_bboxes_scores_ = []
pred_bboxes_list_   = []
for r in range(row_):
    test_img_name = fp_temp_df.iloc[r:r+1]['test_name'].values[0]
    print(test_img_name)
    
    test_img_path = "../../classification/dataset_smooth_186video_20230323/test/0/" + test_img_name               # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    # test_img_path = "../../classification/dataset_186video_20230323/test/0/" + test_img_name
    # test_img_path = "../../classification/dataset_22video_20221031/test/0/" + test_img_name                     # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    
    output_img, detections = model.predict(img_path=test_img_path, 
                                           random_color=False, # 框顏色
                                           plot_img=False, # 顯示出框選圖片
                                           show_text=True, # 框上面加上類別文字
                                           box_num=1,      # 顯示框的數量
                                           return_output=True)
    
    pred_box_num = len(detections['score'])  
               
    if pred_box_num > 0:       # 有預測出box
        score = detections[0:1]['score'].to_numpy()[0]
        pred_bboxes_names_.append(test_img_name)
        pred_bboxes_scores_.append(score)
        pred_bboxes_list_.append(tuple(detections[0:1][['x1','y1','x2','y2']].to_numpy()[0]))

    else:
        pred_bboxes_names_.append(test_img_name)
        pred_bboxes_scores_.append(0.0)
        pred_bboxes_list_.append('NO')
        print('沒有預測出來有box(label)')

In [ ]:
FP_df = pd.DataFrame({"test_name":pred_bboxes_names_, 
                      "pred_bbox":pred_bboxes_list_,
                      "score":pred_bboxes_scores_}
                      )
pred_bboxes = pd.concat([pred_bboxes,FP_df],axis=0,ignore_index=True)
pred_bboxes

In [ ]:
# Left join 
result_table_all              = stage1_pred_result_all.merge(pred_bboxes, on=['test_name'], how='left').merge(true_bboxes, on=['test_name'], how='left')
result_table_all['true_bbox'] = result_table_all.apply(lambda x: 'NO' if (x.test_true==0 and x.stage1_test_pred==1) else x.true_bbox, axis=1)     # FP
result_table_all['pred_bbox'] = result_table_all.apply(lambda x: 'NO' if (x.test_true==1 and x.stage1_test_pred==0) else x.pred_bbox, axis=1)     # FN
result_table_all.shape


In [ ]:
row, col = result_table_all.shape
iou_thresholds = np.arange(0.5, 1.0, 0.05)      

precisions, recalls, f1s = [], [], []
y_trues  = []
y_scores = []
for iou_threshold in iou_thresholds:
    y_true   = []                # 0:無壺腹乳頭 1:有壺腹乳頭
    y_score  = []                # confidence分數
    for r in range(row):         # row: 列數
        test_img_name = result_table_all.iloc[r:r+1]['test_name'][r]
        true_bbox     = result_table_all.iloc[r:r+1]['true_bbox'][r]
        pred_bbox     = result_table_all.iloc[r:r+1]['pred_bbox'][r]
        score         = result_table_all.iloc[r:r+1]['score'][r]

        # print(true_bbox, pred_bbox)
        if true_bbox != 'NO' and pred_bbox != 'NO':
            ciou = get_iou(true_bbox, pred_bbox)           # 找出真實box和預測box的iou
            
            # ciou = calculate_ciou(true_bbox, pred_bbox)      # 找出真實box和預測box的ciou
            
            if ciou >= iou_threshold:   # 如果>=iou，才是有預測到，TP會增加
                y_true.append(1)
                y_score.append(score)   # TP
            else:
                y_true.append(0)
                y_score.append(score)   # FP
        elif true_bbox != 'NO' and pred_bbox == 'NO':
            y_true.append(1)
            y_score.append(0)       # FN
        elif true_bbox == 'NO' and pred_bbox != 'NO':
            y_true.append(0)
            y_score.append(score)       # FP

    y_trues.append(y_true)
    y_scores.append(y_score)

plt_precision_recall_curve_2(iou_thresholds, y_trues, y_scores)     # 畫PR curve

In [ ]:
result_table_all

In [ ]:
# result_table_all.to_csv("../result_df/test_stage1Andstage2_result_smooth_20230331.csv", index=False)
# result_table_all.to_csv("../result_df/test_stage1Andstage2_result_20230331.csv", index=False)